In [1]:
from osgeo import ogr
from osgeo import osr

In [2]:
path_kunming = 'E:\QGIS Data\kunming_district_dtname.shp'

# 坐标转换

#### epsg = 4326
#### utm 32648

In [3]:
path_kunming_utm = 'E:\QGIS Data\convert\kunming_utm.shp'
utm_epsg = 32648
in_ds = ogr.Open(path_kunming)
in_layer = in_ds.GetLayer(0)

driver = ogr.GetDriverByName('ESRI Shapefile')
out_ds = driver.CreateDataSource(path_kunming_utm)
out_layer = out_ds.CreateLayer('kunming_utm',geom_type=ogr.wkbPolygon)
for i in range(in_layer.GetFeature(0).GetFieldCount()):
    field_defn = in_layer.GetFeature(0).GetFieldDefnRef(i)
    out_layer.CreateField(field_defn)

### 空间参考
spatialRef = in_layer.GetSpatialRef()
utm_spatialRef = osr.SpatialReference()
utm_spatialRef.ImportFromEPSG(utm_epsg)

### 投影转换
coordinate_transform = osr.CoordinateTransformation(spatialRef,utm_spatialRef)

### 要素定义
feadefn = in_layer.GetLayerDefn()
### 遍历要素
for i in range(in_layer.GetFeatureCount()):
    in_fea = in_layer.GetFeature(i)
    in_geo = in_fea.geometry()
    in_geo.Transform(coordinate_transform)
    out_geo = in_geo
    out_fea = ogr.Feature(feadefn)
    out_fea.SetGeometry(out_geo)
    for j in range(feadefn.GetFieldCount()):
        field_value = in_fea.GetField(j)
        out_fea.SetField(j,field_value)
    out_layer.CreateFeature(out_fea)

ds = None
out_ds = None

### 保存投影文件
prj_path = path_kunming_utm.replace('.shp','.prj')
fn = open(prj_path,'w')
fn.write(utm_spatialRef.ExportToWkt())

fn.close()





2.添加/删除面积字段

In [5]:
ds = ogr.Open(path_kunming_utm,1)
layer = ds.GetLayer(0)
field_defn = ogr.FieldDefn('area',ogr.OFTReal)
layer.CreateField(field_defn)
ds = None


In [ ]:
ds = ogr.Open(path_kunming_utm,1)
layer = ds.GetLayer(0)
for i in range(layer.GetFeatureCount()):
    fea = layer.GetFeature(i)
    fea_area = fea.geometry().Area()
    fea_area = fea_area/(1000*1000)
    #print('area is',fea_area)
    fea.SetField('area(km2)',fea_area)
    layer.SetFeature(fea)
ds =None

3.获取要素几何中心点

In [7]:
path_kunming_center = 'E:\QGIS Data\convert\kunming_center.shp'
in_ds = ogr.Open(path_kunming,0)
in_layer = in_ds.GetLayer(0)
in_spatialref = in_layer.GetSpatialRef()

driver = ogr.GetDriverByName('ESRI Shapefile')
out_ds = driver.CreateDataSource(path_kunming_center)
out_layer = out_ds.CreateLayer('kunming_center',geom_type=ogr.wkbPoint,srs=in_spatialref)

field_defn = ogr.FieldDefn('fid',ogr.OFTInteger)
out_layer.CreateField(field_defn)
out_fea_defn = out_layer.GetLayerDefn()

for i in range(in_layer.GetFeatureCount()):
    in_fea = in_layer.GetFeature(i)
    out_geo = in_fea.geometry().Centroid()
    out_fea = ogr.Feature(out_fea_defn)
    out_fea.SetGeometry(out_geo)
    out_fea.SetField('fid',i+1)
    out_layer.CreateFeature(out_fea)

out_ds = None
ds = None
